In [8]:
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy

image_asset_path = "./images/"

In [9]:
path = glob( image_asset_path +  '*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filepath in path:

    info = xet.parse(filepath)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filepath)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [10]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)

print(df.head())

filepath = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join(image_asset_path,filename_image)
    return filepath_image

print(getFilename(filepath))

def get_image_path(filepath):
    filename_image = xet.parse(filepath).getroot().find('filename').text
    filepath_image = os.path.join(image_asset_path, filename_image)
    return filepath_image

print(get_image_path(filepath))

# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
df.head()

            filepath  xmin  xmax  ymin  ymax   filename  width  height
0    ./images\N1.xml  1093  1396   645   727    N1.jpeg   1920    1080
1  ./images\N100.xml   134   301   312   350  N100.jpeg    450     417
2  ./images\N101.xml    31   139   128   161  N101.jpeg    249     239
3  ./images\N102.xml   164   316   216   243  N102.jpeg    478     395
4  ./images\N103.xml   813  1067   665   724  N103.jpeg   1800    1200
./images/N1.jpeg
./images/N1.jpeg


,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images\N1.xml,1093,1396,645,727,N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,./images\N100.xml,134,301,312,350,N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,./images\N101.xml,31,139,128,161,N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,./images\N102.xml,164,316,216,243,N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,./images\N103.xml,813,1067,665,724,N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


In [11]:
# make train and test directories
import os
# Define the directories to be created
directories = [
    "data_images/",
    "data_images/test/",
    "data_images/train/"
]
# Create each directory
for directory in directories:
    os.makedirs(directory, exist_ok=True)

In [12]:
def prepare_data(df, folder):
    values = df[['filepath', 'filename', 'center_x', 'center_y', 'bb_width', 'bb_height']].values
    for fpath, fname, x, y, w, h in values:
        image_name = os.path.split(fname)[-1]
        txt_name = os.path.splitext(image_name)[0]
        actual_image_path = getFilename(fpath)
    
        dst_image_path = os.path.join(folder, image_name)
        dst_label_file = os.path.join(folder, txt_name + '.txt')
    
        # copy each image into the folder
        copy(actual_image_path, dst_image_path)

        # generate .txt which has label info
        label_txt = f'0 {x} {y} {w} {h}'
        with open(dst_label_file, mode='w') as f:
            f.write(label_txt)
            f.close()

### split the data into train and test
df_train = df.iloc[:200]
df_test = df.iloc[200:]

# prepare train data
prepare_data(df_train, './data_images/train')

# prepare test data
prepare_data(df_test, './data_images/test')

In [13]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache() 

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 13% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% | 13% |


### To train the model
!python ./yolov5/train.py --data ./msc_anpr/data.yaml --cfg ./yolov5/models/yolov5s.yaml --batch-size 8 --name Model --epochs 100

### Once the model is trained, we can export it to ONNX format
!python ./yolov5/export.py --weight ./yolov5/runs/train/Model/weights/best.pt --include torchscript onnx --opset 12

In [18]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
torch.cuda.get_device_name(0)

True
1


'NVIDIA GeForce RTX 3060 Ti'

In [ ]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

In [ ]:
file_path = image_path[0] #path of our image N1.jpeg
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=1000, height=1000, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 8 - N2.jpeg with bounding box')
fig.add_shape(type='rect',x0=1093, x1=1396, y0=645, y1=727, xref='x', yref='y',line_color='cyan')